[View in Colaboratory](https://colab.research.google.com/github/Sun-Jiankun/TimeSeriesForecasting/blob/master/kaggle_two_sigma_using_kagglegym.ipynb)

In [0]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive')
# !ls "/content/drive/My Drive"

In [0]:

# # pip install kaggle 
# !pip install kaggle

# !cp /content/drive/My\ Drive/kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json




# make kaggle API accessible

# from googleapiclient.discovery import build
# import io, os
# from googleapiclient.http import MediaIoBaseDownload
# from google.colab import auth
# auth.authenticate_user()
# drive_service = build('drive', 'v3')
# results = drive_service.files().list(
#         q="name = 'kaggle.json'", fields="files(id)").execute()
# kaggle_api_key = results.get('files', [])
# filename = "~/.kaggle/kaggle.json"
# os.makedirs(os.path.dirname(filename), exist_ok=True)
# request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
# fh = io.FileIO(filename, 'wb')
# downloader = MediaIoBaseDownload(fh, request)
# done = False
# while done is False:
#     status, done = downloader.next_chunk()
#     print("Download %d%%." % int(status.progress() * 100))
# os.chmod(filename, 600)

# !cp /content/drive/My\ Drive/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!pip install --upgrade tables

In [0]:
import pandas as pd

with pd.HDFStore("./train.h5","r") as train:
    df =  train.get("train")

In [0]:
len(df)

In [0]:
df.head()
df.columns

In [0]:
values = []
labels = []
for col in df.columns:
    labels.append(col)
    values.append(df[col].isnull().sum()) # 记录每一列的缺失值个数
    print(col,values[-1])
    # print(df[col].isnull())

In [0]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

ind = np.arange(len(labels))
width = 0.9
fig, ax = plt.subplots(figsize=(12, 50))
rect = ax.barh(ind, np.array(values), color='y')
ax.set_yticks(ind+(width)/2.)